In [6]:
import os
import polars as pl
from deltalake import DeltaTable
from typing import List

def transform_oxyzo_delta_final(delta_folder_path: str) -> pl.DataFrame:
    """
    Transforms the wide Oxyzo Delta Lake data into a long format. 
    It dynamically finds the EOM header row, handles N blocks, and uses Polars 'unpivot'.
    """
    # 1. Load Delta Lake
    dt = DeltaTable(delta_folder_path)
    df = pl.from_arrow(dt.to_pyarrow_table())

    print(df)

    # 2. Define Constants
    FIXED_COL_COUNT = 5
    METRIC_BLOCK_SIZE = 7
    
    fixed_cols = ["Loan ID", "Loan No.", "Org ID", "Org Name", "Sanction Date"]
    metrics = [
        "Current max DPD", "Lifetime max DPD", "Lifetime max DPD on", 
        "Total Overdue", "Principal Overdue", "EMI Overdue", "POS"
    ]

    # 3. Dynamic Date Row Detection (The Core Fix)
    # We search the first 5 rows for the row containing the EOM date structure.
    date_row_index = -1
    eom_row = None
    
    # The first EOM date should be at index 5. We check up to 5 rows deep.
    for row_idx in range(min(5, df.height)):
        current_row = df.row(row_idx)
        
        # We test if the first metric block's header contains a date-like string
        # We check within the first 3 columns of the block (indices 5, 6, 7)
        for check_idx in range(5, 8):
            val = current_row[check_idx]
            # Check for non-null and date-like content
            if isinstance(val, str) and val.strip() and '-' in val:
                eom_row = current_row
                date_row_index = row_idx
                break
        
        if eom_row is not None:
            break
    
    if eom_row is None:
        raise ValueError("❌ Critical Error: Could not locate the EOM date header row within the first 5 rows of the Delta Lake data.")

    # 4. Calculate N (Number of EOM blocks)
    total_cols = df.width
    
    if (total_cols - FIXED_COL_COUNT) < 0 or (total_cols - FIXED_COL_COUNT) % METRIC_BLOCK_SIZE != 0:
        raise ValueError(f"❌ Invalid column count ({total_cols}). Expected 5 fixed columns plus N * 7 metric columns.")
    
    n_blocks = (total_cols - FIXED_COL_COUNT) // METRIC_BLOCK_SIZE
    
    # 5. Extract EOM Dates resiliently from the found row
    eom_dates: List[str] = []
    
    for i in range(n_blocks):
        start_index = FIXED_COL_COUNT + (i * METRIC_BLOCK_SIZE)
        end_index = start_index + METRIC_BLOCK_SIZE
        
        # Search the entire 7-column block for the date
        block_slice = eom_row[start_index : end_index]
        date_val = None
        
        for val in block_slice:
            if isinstance(val, str) and val.strip() and '-' in val:
                date_val = val.strip()
                break
        
        if date_val is None:
             raise ValueError(f"❌ Date found the date row (index {date_row_index}), but still failed to find date in block {i+1}. Block slice: {block_slice}")
            
        eom_dates.append(date_val)

    # 6. Generate New Column Names
    new_columns = fixed_cols.copy()
    for date in eom_dates:
        for metric in metrics:
            new_columns.append(f"{date}||{metric}")

    # 7. Apply Names and Slice Data
    
    # Remove all rows UP TO AND INCLUDING the date header row
    # If date_row_index is 0, we slice(1). If it's 1, we slice(2).
    df_data = df.slice(date_row_index + 1)
    
    # Apply new column names to the data rows
    df_data = df_data.select(df_data.columns[:len(new_columns)])
    df_data.columns = new_columns

    # 8. Unpivot (formerly melt) - Transform Wide to Long
    df_long_vars = [c for c in new_columns if "||" in c]

    # 

    df_unpivoted = df_data.unpivot(
        index=fixed_cols,
        on=df_long_vars,
        variable_name="EOM_Metric",
        value_name="Value"
    )

    # 9. Split EOM and Metric and Pivot Back (Standard procedure)
    df_long = df_unpivoted.with_columns(
        pl.col("EOM_Metric").str.split_exact("||", 1)
        .struct.rename_fields(["EOM", "Metric"])
        .alias("temp")
    ).unnest("temp").drop("EOM_Metric")

    df_final = df_long.pivot(
        on="Metric",
        index=fixed_cols + ["EOM"],
        values="Value",
        aggregate_function="first",
        sort_columns=True
    )

    # 10. Type Casting
    cols_to_cast = []
    final_order = fixed_cols + ["EOM"] + metrics
    
    for col_name in metrics:
        if col_name == "Lifetime max DPD on":
            cols_to_cast.append(
                pl.col(col_name)
                .str.strip_chars()
                .str.strptime(pl.Date, "%d-%m-%Y", strict=False) 
                .alias(col_name)
            )
        else:
            cols_to_cast.append(
                pl.col(col_name).cast(pl.Float64, strict=False).alias(col_name)
            )

    df_final = df_final.with_columns(cols_to_cast)

    df_final = df_final.select(final_order).sort("Loan No.")

    return df_final.select(final_order)

In [7]:

pl.Config.set_fmt_str_lengths(2000)   # widen column display
pl.Config.set_tbl_width_chars(2000)   # expand table width
pl.Config.set_tbl_rows(50000)         # show up to 50k rows
pl.Config.set_tbl_cols(500) 

def main():
    result = transform_oxyzo_delta_final("E:\\Dev\\Oxyzo R&D\\RndGit\\SourceDeltaLake")

    os.makedirs("E:\\Dev\\Oxyzo R&D\\RndGit\\Output", exist_ok=True)

    result.write_csv("E:\\Dev\\Oxyzo R&D\\RndGit\\Output\\OxyzoDataPreparation_Output.csv")
    print(result)


if __name__ == "__main__":
    main()

shape: (5, 19)
┌─────────┬─────────────┬────────┬──────────┬───────────────┬─────────────────┬──────────────────┬─────────────────────┬───────────────┬───────────────────┬─────────────┬───────────┬──────────────────────────────┬───────────────────────────────┬──────────────────────────────────┬────────────────────────────┬────────────────────────────────┬──────────────────────────┬──────────────────┐
│ Loan ID ┆ Loan No.    ┆ Org ID ┆ Org Name ┆ Sanction Date ┆ Current max DPD ┆ Lifetime max DPD ┆ Lifetime max DPD on ┆ Total Overdue ┆ Principal Overdue ┆ EMI Overdue ┆ POS       ┆ Current max DPD_duplicated_0 ┆ Lifetime max DPD_duplicated_0 ┆ Lifetime max DPD on_duplicated_0 ┆ Total Overdue_duplicated_0 ┆ Principal Overdue_duplicated_0 ┆ EMI Overdue_duplicated_0 ┆ POS_duplicated_0 │
│ ---     ┆ ---         ┆ ---    ┆ ---      ┆ ---           ┆ ---             ┆ ---              ┆ ---                 ┆ ---           ┆ ---               ┆ ---         ┆ ---       ┆ ---                     